In [ ]:
!pip install openai pylcs -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Low Resource


/content/drive/MyDrive/Low Resource


In [ ]:
import openai
import os
import json
import glob
import time
import pylcs
import openai.error
import uuid
import csv
import glob
import openai.error
import helpers
import difflib
import re

In [ ]:
openai.api_key = 'INSERT KEY HERE'

In [ ]:
with open('COVID-QA.json','r') as f:
  full_data = json.load(f)["data"]

In [ ]:
def find_in_context(context, substring):
  substring_lower = substring.lower()
  context_lower = context.lower()
  start = " ".join(substring_lower.split()[0:3])
  end = " ".join(substring_lower.split()[-3:])
  start_idx = context_lower.find(start)
  end_idx = context_lower.find(end)

  if start_idx != -1 and end_idx != -1:
    return context[start_idx:end_idx + len(end)]

  return ""

def process_answers(new_context,curr_answers):
  answers = []
  for i,answer in enumerate(curr_answers):
    if new_context.find(answer) != -1:
      answers.append(answer)
    elif new_context.find(answer[:-1]) != -1:
      answers.append(answer[:-1])
    elif new_context.find(answer[0].lower() + answer[1:]) != -1:
      answers.append(answer[0].lower() + answer[1:])
    elif new_context.find(answer[0].lower() + answer[1:-1]) != -1:
      answers.append(answer[0].lower() + answer[1:-1])
    elif find_in_context(new_context, answer) != "":
      answers.append(find_in_context(new_context, answer))
    elif find_in_context(new_context, answer[:-1]) != "":
      answers.append(find_in_context(new_context, answer[:-1]))
    else:
      answers.append("")

  return answers


def find_answer_context(context, answer, max_context_length=500, num_sentences_before=3, num_sentences_after=4):
    # Strip leading and trailing whitespaces from the answer
    answer = answer.strip()

    # Find the sentence that contains the answer
    if not isinstance(context,str):
      return ""
    context_sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', context)
    target_sentence_index = next((i for i, s in enumerate(context_sentences) if answer.lower() in s.lower()), None)

    # If no sentence contains the answer, return None
    if target_sentence_index is None:
        return None

    # Determine the start and end indices of the context window
    context_start = max(0, target_sentence_index - num_sentences_before)
    context_end = min(len(context_sentences), target_sentence_index + num_sentences_after + 1)

    # Extract the answer-containing context
    answer_context = ' '.join(context_sentences[context_start:context_end])

    return answer_context

def process_string(input_string):
    input_string = input_string.strip("\n")
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', input_string)

    # Calculate the index to extract the bottom 50% of sentences
    bottom_half_index = 3 * len(sentences) // 4

    # Extract the bottom 50% of sentences
    bottom_half_sentences = sentences[bottom_half_index:]

    # Join the sentences back into a single string without spaces between sentences
    result = ' '.join(bottom_half_sentences)

    return result

def extract_qa_pairs(qa):
    # Regular expression patterns to capture questions and answers
    question_pattern = r"(?:Question\s*:\s*|Question\s*\d+\s*:\s*)(.*?)(?=\n\s*(?:Question\s*:\s*|Question\s*\d+\s*:|Answer\s*:\s*|Answer\s*\d+\s*:|$))"
    answer_pattern = r"(?:Answer\s*:\s*|Answer\s*\d+\s*:\s*)(.*?)(?=\n\s*(?:Question\s*:\s*|Question\s*\d+\s*:|Answer\s*:\s*|Answer\s*\d+\s*:|$))"

    # Extract questions and answers using regular expressions
    questions = re.findall(question_pattern, qa, flags=re.DOTALL)
    answers = re.findall(answer_pattern, qa, flags=re.DOTALL)

    return questions, answers

In [ ]:
context_multiplier = 4
qa_per_context = 20
total_articles = 116
starting = 85
curr_file = "COVID_AUGMENT_SMALL_TWO_SHOT"

In [ ]:
with open('COVID_AUGMENT_SMALL_TWO_SHOT.json','r') as f:
  dataset = json.load(f)

In [ ]:
import time
import openai

def make_openai_request(prompt: str, model: str, max_tokens=8000):
    max_retry_count = 10
    retry_count = 1
    retry_time_sec = 30

    while retry_count <= max_retry_count:
        try:
            response = openai.ChatCompletion.create(
                model=model, messages=prompt)
            answer = response["choices"][0]["message"]["content"]
            # answer = answer.strip().strip('"')
            return answer
        except openai.error.APIConnectionError as e:
            print(
                f"API connection error: {e}.\nRetrying in {retry_time_sec}...")
        except openai.error.RateLimitError as e:
            print(
                f"Rate limit error: {e}.\nRetrying in {retry_time_sec}...")
        except openai.error.Timeout as e:
            print(f"Timeout error: {e}.\nRetrying in {retry_time_sec}...")
        except openai.error.ServiceUnavailableError as e:
            print(
                f"ServiceUnavailable error: {e}.\nRetrying in {retry_time_sec}...")
        except OSError as e:
            from google.colab import drive
            drive.mount('/content/drive')
            %cd drive/MyDrive/Low Resource
        except openai.error.OpenAIError as e:
            print(f"Unhandled OpenAI error occurred: {e}")
        except Exception as e:
            print(f"Unhandled error occurred: {e}")
            break

        retry_count += 1
        time.sleep(retry_time_sec)

    if retry_count > max_retry_count:
        print(
            f"Retries of {retry_count} exceeded max_retry_count of {max_retry_count}")

In [ ]:
def get_two_shot(dataset):
    two_shot = []

    # Iterate through the dataset
    for i, data in enumerate(dataset["data"]):
      if i == len(dataset["data"]) - 1:
        break
      data2 = dataset["data"][i+1]
      context1 = data["paragraphs"][0]["context"]
      question1 = data["paragraphs"][0]["qas"][0]["question"]
      answer1 = data["paragraphs"][0]["qas"][0]["answers"][0]["text"]

      context2 = data2["paragraphs"][0]["context"]
      question2 = data2["paragraphs"][0]["qas"][0]["question"]
      answer2 = data2["paragraphs"][0]["qas"][0]["answers"][0]["text"]

      two_shot.append((context1, question1, answer1, context2, question2, answer2))

    return two_shot


In [ ]:
two_shot = get_two_shot(dataset)


In [ ]:
for cnt in range(starting,total_articles):
  print(f'On document {cnt + 1}/{total_articles}')
  example_context1,example_question1,example_answer1,example_context2,example_question2,example_answer2 = two_shot[cnt]
  short_context1 = process_string(example_context1)
  short_context2 = process_string(example_context2)

  for i in range(context_multiplier - 1):
    new_dict = {"qas":[],"context":""}
    context_prompt = [
                      {"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": f"Generate a context that is similar in topic and domain distribution to the given two context. Example Context:{short_context1}. Example Context:{short_context2} Context:"}
                      ]
    generated_context = make_openai_request(context_prompt, "gpt-4")
    new_dict["context"] = generated_context
    downsized_context1 = find_answer_context(example_context1,example_answer1)
    example_context1 = downsized_context1 if downsized_context1 != "" else example_context1
    downsized_context2 = find_answer_context(example_context2,example_answer2)
    example_context2 = downsized_context2 if downsized_context2 != "" else example_context2


    qa_prompt = [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Based only on the given context generate 1 question answer pair. The answer must be only made up of substrings from the context and do not generate any new text for the answer. Context:{example_context1}."},
                    {"role": "assistant", "content": f"Question: {example_question1} Answer: {example_answer1}"},
                    {"role": "user", "content": f"Based only on the given context generate 1 question answer pair. The answer must be only made up of substrings from the context and do not generate any new text for the answer. Context:{example_context2}."},
                    {"role": "assistant", "content": f"Question: {example_question2} Answer: {example_answer2}"},
                    {"role": "user", "content": f"Based only on the given context generate {qa_per_context} question answer pair. The answer must be only made up of substrings from the context and do not generate any new text for the answer. Context:{generated_context}."}
                ]
    qa = make_openai_request(qa_prompt, "gpt-4")
    if not qa:
      continue
    if len(qa) > 0 and qa[-1] != "\n":
      qa = qa + "\n"
    questions,answers = extract_qa_pairs(qa)
    answers = process_answers(generated_context,answers)
    print(f'    -------->{len([elem for elem in answers if elem != ""])} answers kept')


    if len(answers) != len(questions):
      print("      -------> Skipped this passage")
      continue

    for j in range(len(questions)):
      question = questions[j]
      answer = answers[j]
      ques_id = str(uuid.uuid4())
      ans_start = generated_context.find(answer)

      if answer == "" or ans_start == -1:
        continue

      new_dict["qas"].append({"question":question, "id":ques_id, "answers":[{"text":answer, "answer_start": ans_start}], "is_impossible": False})

    dataset["data"].append({"paragraphs":[new_dict]})
    try:
      with open(f"{curr_file}.json", "w") as f:
        json.dump(dataset, f)
        print("    -------> Saved to Json")
    except OSError as e:
          from google.colab import drive
          drive.mount('/content/drive')
          %cd drive/MyDrive/Low Resource




On document 86/116
    -------->12 answers kept
    -------> Saved to Json
    -------->12 answers kept
    -------> Saved to Json
    -------->15 answers kept
    -------> Saved to Json
On document 87/116
    -------->17 answers kept
    -------> Saved to Json
    -------->15 answers kept
    -------> Saved to Json
    -------->17 answers kept
    -------> Saved to Json
On document 88/116
    -------->20 answers kept
    -------> Saved to Json
    -------->14 answers kept
    -------> Saved to Json
    -------->20 answers kept
    -------> Saved to Json
On document 89/116
    -------->15 answers kept
    -------> Saved to Json
    -------->16 answers kept
    -------> Saved to Json
    -------->13 answers kept
    -------> Saved to Json
On document 90/116
    -------->15 answers kept
    -------> Saved to Json
    -------->16 answers kept
    -------> Saved to Json
    -------->15 answers kept
    -------> Saved to Json
On document 91/116
    -------->16 answers kept
    -------> Save